In [3]:
from tools import get_input
import numpy as np

data, data_test = get_input(19)
len(data), len(data_test)

(744, 17)

### Part 1

In [4]:
def prepare(data):
    sep = [i for i, line in enumerate(data) if len(line)==0][0]
    system = data[:sep]
    messages = data[sep+1:]

    list_messages = []
    for m in messages:
        d = {}
        for s in m[1:-1].split(","):
            k, v = s.split("=")
            d[k] = int(v)
        list_messages.append(d)
    
    d_system = {}
    for wf in system:
        name = wf.split("{")[0].strip()
        rules = wf.split("{")[1][:-1].split(",")[:-1]
        default = wf.split("{")[1][:-1].split(",")[-1]
        d_system[name] = (rules, default)
    
    return d_system, list_messages

def pass_message(m, d):
    cond = True
    engine = "in"
    while cond:
        rules, default = d[engine]
        for rule in rules:
            cond, res = rule.split(":")
            c = cond[0]
            cond = f"m['{c}']" + cond[1:]
            if eval(cond):
                engine = res
                break
        else:
            engine = default
        #print(engine, end=' ')
        cond = engine not in {"A", "R"}
    return engine

d_system, list_messages = prepare(data)
S = 0
for m in list_messages:
    if pass_message(m, d_system)=='A':
        S += sum(m.values())
S

350678

### Part 2

In [ ]:
def split_list(xlist, cond, val):
    '''
    function that splits a list of intervals into two lists
    of intervals, one that satisfies the condition and one that does not
    '''

    list_true = []
    list_false = []
    if cond == ">":
        for (x1, x2) in xlist:
            if x1 > val:
                list_true.append((x1, x2))
            elif x2 > val:
                list_true.append((val+1, x2))
                list_false.append((x1, val))
            else:
                list_false.append((x1, x2))
    elif cond == "<":
        for (x1, x2) in xlist:
            if x2 < val:
                list_true.append((x1, x2))
            elif x1 < val:
                list_true.append((x1, val-1))
                list_false.append((val, x2))
            else:
                list_false.append((x1, x2))
    return list_true, list_false

In [19]:
def count_accepted(node, x_list, m_list, a_list, s_list, d):
    if node=='A':
        nx = sum([x[1]-x[0]+1 for x in x_list])
        nm = sum([m[1]-m[0]+1 for m in m_list])
        na = sum([a[1]-a[0]+1 for a in a_list])
        ns = sum([s[1]-s[0]+1 for s in s_list])
        return nx*nm*na*ns
    elif node=='R':
        return 0
    else:
        res = 0
        rules, default = d[node]
        for rule in rules:
            cond, nextnode = rule.split(":")
            c = cond[0]
            comp = cond[1]
            val = int(cond[2:])
            if c=='x':
                x_true, x_list = split_list(x_list, comp, val)
                res += count_accepted(nextnode, x_true, m_list, a_list, s_list, d)
            elif c=='m':
                m_true, m_list = split_list(m_list, comp, val)
                res += count_accepted(nextnode, x_list, m_true, a_list, s_list, d)
            elif c=='a':
                a_true, a_list = split_list(a_list, comp, val)
                res += count_accepted(nextnode, x_list, m_list, a_true, s_list, d)
            elif c=='s':
                s_true, s_list = split_list(s_list, comp, val)
                res += count_accepted(nextnode, x_list, m_list, a_list, s_true, d)
        res += count_accepted(default, x_list, m_list, a_list, s_list, d)
        return res    
            
N = 4000
x_list = [(1, N)]
m_list = [(1, N)]
a_list = [(1, N)]
s_list = [(1, N)]

node = 'in'

d_system, _ = prepare(data)

count_accepted(node, x_list, m_list, a_list, s_list, d_system)

124831893423809